In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import scipy as sp
from collections import defaultdict
from itertools import product, permutations
import gurobipy as gb
from gurobipy import GRB

repeats = json.load(open('../cleanerdata/repeats.json'))
repeats = {int(k):v for k,v in repeats.items()}

locdem = pd.read_excel('../cleanerdata/locdem.xlsx')

q = locdem['Number of pallets'].to_numpy().astype(float).tolist()
q = [0] + q
q.extend([v['dem'] for v in repeats.values()])

locs = pd.read_excel('../cleanerdata/locations.xlsx')
longs, lats = locs['long'].to_numpy().tolist(), locs['lat'].to_numpy().tolist()
longs.extend([longs[v['map']] for v in repeats.values()])
lats.extend([lats[v['map']] for v in repeats.values()])

distmat = pd.read_json('../cleanerdata/distmat.json').to_numpy()
timemat = pd.read_json('../cleanerdata/timemat.json').to_numpy()
def get(i):
    try:
        i = repeats[i]['map']
    except KeyError:
        pass
    return i

def cost(i,j):
    i,j = get(i), get(j)
    return distmat[i,j]
def time(i,j):
    i,j = get(i), get(j)
    return timemat[i,j]

def location(i):
    return longs[i], lats[i]

In [2]:
loops = json.load(open('./loops.json'))
loops
Loops = []
for loop in loops:
    L = []
    for l in loop:
        if l in repeats:
            L.append(repeats[l]['map'])
        else:
            L.append(l)
    Loops.append(L)
loops = Loops
loops

[[0, 1, 45, 101, 44, 16, 0],
 [0, 59, 0],
 [0, 102, 47, 7, 48, 0],
 [0, 12, 123, 0],
 [0, 88, 86, 2, 0],
 [0, 12, 66, 89, 95, 19, 0],
 [0, 46, 61, 60, 3, 0],
 [0, 51, 49, 0],
 [0, 69, 120, 4, 40, 0],
 [0, 64, 118, 77, 5, 0],
 [0, 111, 34, 5, 0],
 [0, 5, 0],
 [0, 111, 5, 0],
 [0, 40, 108, 35, 6, 0],
 [0, 115, 26, 40, 0],
 [0, 40, 108, 6, 0],
 [0, 57, 72, 98, 97, 0],
 [0, 92, 0],
 [0, 98, 27, 8, 71, 97, 0],
 [0, 32, 0],
 [0, 107, 32, 9, 106, 0],
 [0, 32, 9, 113, 106, 0],
 [0, 32, 0],
 [0, 10, 17, 56, 63, 0],
 [0, 55, 122, 0],
 [0, 38, 122, 21, 62, 0],
 [0, 75, 110, 31, 33, 0],
 [0, 84, 33, 0],
 [0, 110, 11, 81, 33, 0],
 [0, 13, 74, 20, 43, 0],
 [0, 85, 43, 0],
 [0, 43, 0],
 [0, 43, 25, 99, 0],
 [0, 42, 73, 0],
 [0, 42, 14, 78, 0],
 [0, 42, 118, 0],
 [0, 42, 94, 96, 0],
 [0, 41, 117, 0],
 [0, 41, 82, 79, 104, 0],
 [0, 117, 15, 41, 104, 0],
 [0, 114, 38, 116, 50, 0],
 [0, 38, 116, 0],
 [0, 38, 18, 109, 114, 0],
 [0, 22, 119, 0],
 [0, 93, 28, 33, 6, 0],
 [0, 6, 103, 87, 119, 0],
 [0, 100, 7

In [3]:
h = np.zeros(len(loops), dtype=float)
for i, loop in enumerate(loops):
    for j in range(0, len(loop)-1):
        arc = loop[j], loop[j+1]
        h[i] += time(*arc)
h = h / 3600

In [4]:
D = range(16)
L = range(0, len(loops))
T = range(5)
f = 3

m = gb.Model()
z = m.addVars(D, vtype=GRB.BINARY, name='z')
y = m.addVars(L,T,D,vtype=GRB.BINARY, name='y')

m.setObjective(z.sum('*'), GRB.MINIMIZE)
'''for l in L:
    expr = 0
    for t in T:
        for d in D:
            expr += y[l,t,d]
    m.addConstr(expr == 1)'''
    
m.addConstrs((y.sum(l,'*', '*') == 1 for l in L))
m.addConstrs((
    sum([h[l]*y[l,t,d] for l in L]) <= 8 for t in T for d in D
))
m.addConstrs((y[l,t,d] <= z[d] for l in L for t in T for d in D))

for pair in permutations(T, 2):
    t1, t2 = pair
    m.addConstrs((
        sum([h[l]*y[l,t1,d] for l in L]) - sum([h[l]*y[l,t2,d] for l in L]) <= f for d in D
    ))

node_to_loops = defaultdict(list)
for node in range(1,181):
    for i,loop in enumerate(loops):
        if node in loop:
            node_to_loops[node].append(i)
    
doubles = filter(lambda lst: len(lst) == 2, node_to_loops.values())
triples = filter(lambda lst: len(lst) == 3, node_to_loops.values())
quadruples = filter(lambda lst: len(lst) == 4, node_to_loops.values())

for t1, t2 in product(T,T):
    for looops in doubles:
        for d1,d2 in product(D, D):
            if abs(d1 - d2) <= 8:
                for l1,l2 in product(looops, looops):
                    m.addConstr(y[l1,t1,d1] + y[l2,t2,d2] <= 1)

for t1, t2 in product(T,T):
    for looops in quadruples:
        for d1,d2 in product(D, D):
            if abs(d1 - d2) <= 4:
                for l1,l2 in product(looops, looops):
                    m.addConstr(y[l1,t1,d1] + y[l2,t2,d2] <= 1)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-12


In [5]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 45068 rows, 4816 columns and 135808 nonzeros
Model fingerprint: 0x1f64c1e0
Variable types: 0 continuous, 4816 integer (4816 binary)
Coefficient statistics:
  Matrix range     [3e-01, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 16.0000000
Presolve removed 43648 rows and 736 columns
Presolve time: 0.12s
Presolved: 1420 rows, 4080 columns, 14816 nonzeros
Variable types: 64 continuous, 4016 integer (4016 binary)

Root relaxation: objective 1.000000e+00, 2306 iterations, 0.10 seconds (0.10 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.00000    0  189   16.00000    1.00000  93.8%     -    0s
H    0 

In [6]:
soln = json.loads(m.getJSONSolution())

In [7]:
soln;

In [8]:
Y = np.array(list(map(lambda var: var.x, y.values()))).reshape(len(L), len(T), len(D))
Y = Y.astype(int)
Y.tolist()

[[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 

In [9]:
json.dump(Y.tolist(), open('./finalroutings.json', 'w'))